In [46]:
import numpy as np
import scipy
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics.pairwise import pairwise_kernels
from scipy.optimize import minimize
from scipy.spatial.distance import hamming

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('Data/train.csv')

In [38]:
images = ["%s%s" %("pixel",pixel_no) for pixel_no in range(0,28**2)]
train_images = np.array(train[images], dtype=np.float)/100
#dimensionserhoehung, damit am ende b = 0
train_images = np.concatenate((train_images, np.ones((len(train_images),1))), axis = 1)

train_labels = np.array(train['label'])
label0 = np.array([1 if i==0 else -1 for i in train_labels])
len(train_images)

42000

In [62]:
number_of_im = 200;
train_images_test = train_images[:number_of_im]
train_labels_test = train_labels[:number_of_im]
np.shape(train_images_test)[0]


200

In [82]:
from mySVM import *


ModuleNotFoundError: No module named 'mySVM'

In [63]:
def ecoc(labeled_data, labels, kernel=scalar_product, penalty=1, list_sigma=[0.1]*15):
    # 
    labels=labels.astype(int);
    l=np.shape(labeled_data)[0];
    num_classifiers=15;
    ecoc_labels=np.zeros((l,15));
    code_words=np.array([
        [ 1,  1, -1, -1, -1, -1,  1, -1,  1, -1, -1,  1,  1, -1,  1],
        [-1, -1,  1,  1,  1,  1, -1,  1, -1,  1,  1, -1, -1,  1, -1],
        [ 1, -1, -1,  1, -1, -1, -1,  1,  1,  1,  1, -1,  1, -1,  1],
        [-1, -1,  1,  1, -1,  1,  1,  1, -1, -1, -1, -1,  1, -1,  1],
        [ 1,  1,  1, -1,  1, -1,  1,  1, -1, -1,  1, -1, -1, -1,  1],
        [-1,  1, -1, -1,  1,  1, -1,  1,  1,  1, -1, -1, -1, -1,  1],
        [ 1, -1,  1,  1,  1, -1, -1, -1, -1,  1, -1,  1, -1, -1,  1],
        [-1, -1, -1,  1,  1,  1,  1, -1,  1, -1,  1,  1, -1, -1,  1],
        [ 1,  1, -1,  1, -1,  1,  1, -1, -1,  1, -1, -1, -1,  1,  1],
        [-1,  1,  1,  1, -1, -1, -1, -1,  1, -1,  1, -1, -1,  1,  1]]);
    for j in range(l):
        ecoc_labels[j]=code_words[labels[j]];
    
    list_supp_ind = [];
    list_alpha =[];
    list_b =[];
    list_kernel=[];
    
    for classifier in range(15):
        svm=mySVM(kernel=kernel, penalty=penalty, sigma=list_sigma[classifier]);
        svm.fit(labeled_data, ecoc_labels[:,classifier]);
        list_supp_ind.append(svm.supp_indices);
        list_alpha.append(svm.alpha);
        list_b.append(svm.b);
        list_kernel.append(svm.kernel);
        
    # pickle dump / pickle load (?) to save and call saved objects    
    
    # now I need to call a binary classifier for each column of ecoc_labels
    # from decision functions we get seperating hyperplanes, margin, ... 
    # return those
    
    
    return ecoc_labels, list_supp_ind, list_alpha, list_b, list_kernel, code_words;

In [64]:
ecoc_labels, list_supp_ind, list_alpha, list_b, list_kernel, code_words=ecoc(train_images_test, train_labels_test)

In [58]:
# suppose we have an unlabeled data point
def predict_ecoc(unlabeled_data, labeled_data, ecoc_labels, list_supp_ind, list_alpha, list_b, list_kernel, code_words):
    # every row is one data point
    # number of rows = # of data points
    l=np.shape(unlabeled_data)[0];
    print(l)
    new_labels=np.zeros((l,15));
    
    temp_label_ind=[];
    final_labels = np.array(['NaN']*l);
    
    for classifier in range(15):
        a_supp = list_alpha[classifier][list_supp_ind[classifier]];
        ecoc_labels_supp = ecoc_labels[list_supp_ind[classifier],classifier]
        a_times_labels=np.multiply(a_supp, ecoc_labels_supp)
        
        for i in range(l):
            # i_th row of kernel matrix k
            k=np.array([list_kernel[classifier](unlabeled_data[i],y) for y in labeled_data[list_supp_ind[classifier]]])
            
            # list of lists with 15 entries, one per classifier
            new_labels[i][classifier]=np.sign(np.dot(a_times_labels,k)+list_b[classifier]);
    
        
    for i in range(l):
        ham_dist = [hamming(new_labels[i], code_words[j]) for j in range(10)]
        temp_label_ind = [j for j in range(len(ham_dist)) if ham_dist[j] == min(ham_dist)]
        if len(temp_label_ind)!=1:
            print("Attention, data point could not be uniquely classified, index " 
                  + str(i) + "possible classification" + str(temp_label_ind));
        else:
            final_labels[i] = ham_dist.index(min(ham_dist));
        
   
    return final_labels;
    

array([5, 1, 9, 0], dtype=int64)

In [81]:
# for testing
predicted_labels_test=predict_ecoc(train_images[number_of_im:number_of_im+20], train_images[:number_of_im], ecoc_labels, list_supp_ind, list_alpha, list_b, list_kernel, code_words)
print("predicted labels: ", predicted_labels_test)
train_labels_test=train_labels[number_of_im:number_of_im+20]
print(type(train_labels_test[0]))
print(type(predicted_labels_test[0]))
sum(predicted_labels_test.astype(int)==train_labels_test.astype(int))

20
predicted labels:  ['0' '0' '6' '0' '0' '4' '9' '2' '7' '1' '9' '9' '3' '9' '8' '2' '6' '4'
 '1' '3']
<class 'numpy.int64'>
<class 'numpy.str_'>


16

In [19]:
def gaussian_kernel(x, y, sigma):
	return np.exp(-sigma*np.linalg.norm(x-y)**2)

class mySVM:
    from smo import smo
    
    def __init__(self, kernel = gaussian_kernel, penalty = 1, sigma = 0.01):
        self.sigma = sigma
        self.kernel = kernel
        self.penalty = penalty
        self.kernelmatrix_for_predict = None
        self.tolerance = 1e-5
        
    def fit(self,training_data, training_labels):
        kernel_identifier = None
        if self.kernel == gaussian_kernel:
            def kernel_sigma(x, y):
                return gaussian_kernel(x, y, self.sigma)
            self.kernel = kernel_sigma
        if self.kernel == scalar_product:
            kernel_identifier = 'standard scalar product'
        solution = smo(np.transpose(training_data),training_labels,self.penalty,self.kernel,self.tolerance,'yes', kernel_identifier)
        self.alpha = solution['solution']
        self.training_data = training_data
        self.training_labels = training_labels
        self.supp_indices, self.supp_vectors = extract_suppvectors(self.alpha, training_data) #support vectors and indices of support vectors
        self.b = extract_b(self.alpha[self.supp_indices], training_labels[self.supp_indices], self.supp_vectors, self.kernel, self.penalty)
        if self.kernel == scalar_product:
            self.w = extract_w(self.alpha, training_labels, training_data, self.supp_indices)
			
def extract_suppvectors(alpha, data):
    supp_indices = np.array([i for i in range(len(data)) if not alpha[i]==0])
    supp_vectors = data[supp_indices]
    return supp_indices, supp_vectors

def extract_w(alpha, labels, data, supp_indices):
    #nur für linear kernels
    w = np.sum((np.array(alpha[i]*labels[i]*data[i]) for i in supp_indices), axis = 0)
    return w

def y_withoutb(x, alpha_supp, labels_supp, data_supp, kernel):
    alphatimeslabels = np.multiply(alpha_supp, labels_supp)
    k = np.array([kernel(x, y) for y in data_supp])
    return np.dot(alphatimeslabels, k)

def extract_b(alpha_supp, labels_supp, data_supp, kernel, C):
    indicesonmargin = np.array([i for i in range(len(data_supp)) if alpha_supp[i]<C ])
    return 1./len(indicesonmargin)*sum([labels_supp[i]-y_withoutb(data_supp[i],alpha_supp, labels_supp, data_supp, kernel) for i in indicesonmargin])
	
def scalar_product(v1,v2):
    '''standard scalar product'''
    return np.dot(v1,v2)
	


In [13]:
import numpy as np

# careful, smo expects data in the shape of a dxl array, where d is the dimension of the data points and l the number of data points!
def smo(data,label,C,kernel,tol,violationcheckyesorno, kernel_identifier = None):
    
    eps = 1e-12
    
    l = label.shape[0]
    alpha = np.zeros(l)
    
    fcache = -label.astype(float)
    
    b_up = [-1]
    b_low = [1]
    
    I = np.zeros((5,l), dtype=np.int)
    I[1,:] = (label == 1).astype(int)
    I[4,:] = (label == -1).astype(int)

    i_up = [min(np.nonzero(I[1,:])[0])]
    i_low = [min(np.nonzero(I[4,:])[0])]
    
    
    ##############################################################################################
    # auxiliary functions for takeStep and examineExample
    
    def I_membership(a,y):
        if 0 < a and a < C:
            return np.array([1,0,0,0,0])
        elif a == 0 and y == 1:
            return np.array([0,1,0,0,0])
        elif a == C and y == -1:
            return np.array([0,0,1,0,0])
        elif a == C and y == 1:
            return np.array([0,0,0,1,0])
        else:
            return np.array([0,0,0,0,1])
    

    def I_membership_no(a,y):
        if 0 < a < C:
            return 0
        elif a == 0 and y == 1:
            return 1
        elif a == C and y == -1:
            return 2
        elif a == C and y == 1:
            return 3
        else:
            return 4
        
    def F(i,alpha):
        
        #avoid for loop in case of standard scalar product kernel
        if kernel_identifier == 'standard scalar product':
            k = alpha * label;
            m = np.dot(data[:,i],data)
            out = -label[i] + np.dot(k,m)
        else: 
            out = - label[i]
            for j in range(l):
                out += alpha[j]*label[j]*kernel(data[:,i],data[:,j])
        return out
    
    
    ##############################################################################################   
    
    
    
    
    ##############################################################################################
    # subprocedure takeStep
    
    def takeStep(i1,i2):

    
        if i1 == i2: 
            return 0
        alph1 = alpha[i1]
        alph2 = alpha[i2]
        y1 = label[i1]
        y2 = label[i2]  
   
        F1 = fcache[i1]
        F2 = fcache[i2]
        s = y1*y2
    
        if s == -1:
            L = max(0,alph2-alph1)
            H = min(C,C+alph2-alph1)
        else:
            L = max(0,alph2+alph1-C)
            H = min(C,alph2+alph1)
    
        if L == H: return 0
    
        k11 = kernel(data[:,i1],data[:,i1])
        k12 = kernel(data[:,i1],data[:,i2])
        k22 = kernel(data[:,i2],data[:,i2])
    
        eta = 2*k12 - k11 - k22
        if eta < 0:
            a2 = alph2 - y2*(F1-F2)/eta
            if a2 < L:
                a2 = L
            elif a2 > H:
                a2 = H
        else:
            #print('Error: eta == 0')
            raise ValueError('Error: eta == 0')
    
        if abs(a2-alph2) < eps*(a2+alph2+eps):
            return 0
    
        a1 = alph1 + s*(alph2-a2)
    
   
        # update alpha
        alpha[i1] = a1
        alpha[i2] = a2
    
        # update I
        I[:,i1] =  I_membership(a1,y1)
        I[:,i2] =  I_membership(a2,y2)
    
        # update fcache[i] here for i in I_0: a[0:3][:,4:9]
        ind_0 = (np.nonzero(I[0,:]))[0]
        
        #avoid for loop in case of standard scalar product kernel
        if kernel_identifier == 'standard scalar product':
            fcache[ind_0] += y1*(a1-alph1)*np.dot(data[:,i1],data[:,ind_0]) + y2*(a2-alph2)*np.dot(data[:,i2],data[:,ind_0])
        else:
            for i in ind_0:
                fcache[i] = fcache[i] + y1*(a1-alph1)*kernel(data[:,i],data[:,i1]) + y2*(a2-alph2)*kernel(data[:,i],data[:,i2])
    
        # update fcache for indices i1 and i2
        fcache[i1] = F1 + y1*(a1-alph1)*k11 + y2*(a2-alph2)*k12
        fcache[i2] = F2 + y1*(a1-alph1)*k12 + y2*(a2-alph2)*k22
        
        
        b_low[0] = -float('inf')
        b_up[0] = - b_low[0]
        
        if np.size(ind_0) > 0:
            for i in ind_0:
                if fcache[i] > b_low[0]:
                    b_low[0] = fcache[i]
                    i_low[0] = i
                if fcache[i] < b_up:
                    b_up[0] = fcache[i]
                    i_up[0] = i
       
        memb1 = I_membership_no(a1,y1)
        memb2 = I_membership_no(a2,y2)
        if memb1 in (1,2) and fcache[i1] < b_up[0]:
            b_up[0] = fcache[i1]
            i_up[0] = i1
        if memb1 in (3,4) and fcache[i1] > b_low[0]:
            b_low[0] = fcache[i1]
            i_low[0] = i1
        if memb2 in (1,2) and fcache[i2] < b_up[0]:
            b_up[0] = fcache[i2]
            i_up[0] = i2
        if memb2 in (3,4) and fcache[i2] > b_low[0]:
            b_low[0] = fcache[i2]
            i_low[0] = i2

        return 1
    
    ##############################################################################################
    
    
    
    
    ##############################################################################################
    #subprocedure examineExample
    
    def examineExample(i2):

    
        y2 = label[i2]
        alph2 = alpha[i2]
        memb_i2 = I_membership_no(alph2,y2);
    
        # look up or compute F2:= F_i2
        if memb_i2 == 0:
            F2 = fcache[i2]
        else:
            fcache[i2] = F(i2,alpha)
            F2 = fcache[i2]
        
            # in case i2 not in I_0, see if F2 changes or estimate of b_low or b_up
            if memb_i2 in (1,2) and F2 < b_up[0]:
                b_up[0] = F2
                i_up[0] = i2
            elif memb_i2 in (3,4) and F2 > b_low[0]:
                b_low[0] = F2
                i_low[0] = i2
    
        optimality = 1
        if memb_i2 in (0,1,2):
            if b_low[0] - F2 > tol:
                optimality = 0
                i1 = i_low[0]
        if memb_i2 in (0,3,4):
            if F2 - b_up[0] > tol:
                optimality = 0
                i1 = i_up[0]
        if optimality == 1:
            return 0
    
        if memb_i2 == 0:
            if b_low[0] - F2 > F2 - b_up[0]:
                i1 = i_low[0]
            else:
                i1 = i_up[0]
            
        if takeStep(i1,i2):
            return 1
        else:
            return 0
    
    ##############################################################################################
    
    
    
    
    ##############################################################################################
    #main routine
    
    numChanged = 0
    examineAll = 1

    while numChanged > 0 or examineAll:
        numChanged = 0
        if examineAll:
            for i in range(l):
                numChanged = numChanged + examineExample(i)
        else:
            for i in (np.nonzero(I[0,:]))[0]:
                numChanged = numChanged + examineExample(i)
                if b_up[0] > b_low[0] - tol:
                    numChanged = 0
                    break
        if examineAll == 1:
            examineAll = 0
        elif numChanged == 0:
            examineAll = 1
            
            
    if violationcheckyesorno == 'yes':
        memb = [I_membership_no(alpha[i],label[i]) for i in range(l)]
        I_up = [memb[i] in (0,1,2) for i in range(l)]
        I_low = [memb[i] in (0,3,4) for i in range(l)]
        bs = [F(i,alpha) for i in range(l)]

        b_up = float('inf')
        b_low = -b_up
        for i in range(l):
            if I_up[i] == 1 and bs[i] < b_up:
                b_up = bs[i]
            if I_low[i] == 1 and bs[i] > b_low:
                b_low = bs[i]

        if b_low - tol <= b_up:
            violationstring = 'no tol-violation'
        else:
            violationstring = 'tol-violations!!'
    else:
        violationstring = 'no violation requested'

    
    return {'solution': alpha, 'violationcheck': violationstring}

    ##############################################################################################